#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter [DV-3PO] Custom Signals Parameters
[DV-3PO] Custom Signals allows automated changes to be made to DV360 campaigns based on external signals from weather and social media trends. In the future it will also support news, disaster alerts, stocks, sports, custom APIs, etc. 
 1. Open the template sheet: <a target='_blank' href='https://docs.google.com/spreadsheets/d/1xl2bknsn9ptCKwg1sZqTBpmRFDdcG3ATdp2f4C2CQ2U/edit#gid=1579485492'>[DV-3PO] Custom Signals Configs</a>.
 1. Make a copy of the sheet through the menu File -> Make a copy, for clarity we suggest you rename the copy to a meaningful name describing the usage of this copy.
 1. In the Station IDs field below enter a comma separated list of NOAA weather station IDs. Most major airports are stations and their ID typically is K followed by the 3 letter airport code, e.g. KORD for Chicago O'Hare International Airport, KSFO for San Francisco international airport, etc. You can get a full list of stations <a target='_blank' href='https://www1.ncdc.noaa.gov/pub/data/noaa/isd-history.txt'>here</a>, the station ID to use is the 'CALL' column of this list.
 1. In the Sheet URL field below, enter the URL of the copy of the config sheet you've created.
 1. Go to the sheet and configure the rules you'd like to be applied in the Rules tab.
 1. In the Advertiser ID column, enter the advertiser ID of the line items you'd like to automatically update.
 1. In the Line Item ID colunn, enter the line item ID of the line item you'd like to automatically update.
 1. The 'Active' column of the Rules tab allows you to control if the line item should be active or paused. If this field is TRUE the line item will be set to active, if this field is FALSE the line item will be set to inactive. You can use a formula to take weather data into consideration to update this field, e.g. =IF(Weather!C2>30, TRUE, FALSE) will cause the line item to be activated if the temperature of the first station in the Weather tab is above 30 degrees. Leave this field empty if you don't want it to be modified by the tool.
 1. The 'Fixed Bid' column of the Rules tab allows you to control the fixed bid amount of the line item. The value set to this field will be applied to the specified line item. You can use a formula to take weather data into consideration to update this field, e.g. =IF(Weather!G2>3, 0.7, 0.4) will cause bid to be set to $0.7 if the wind speed of the first line in the Weather tab is greater than 3 mph, or $0.4 otherwise. Leave this field empty if you don't want it to be modified by the tool.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'station_ids': '',  # NOAA Weather Station ID
  'auth_read': 'user',  # Credentials used for reading data.
  'sheet_url': '',  # Feed Sheet URL
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute [DV-3PO] Custom Signals
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'weather_gov': {
      'auth': 'user',
      'stations': {'field': {'name': 'station_ids','kind': 'string_list','order': 1,'description': 'NOAA Weather Station ID','default': ''}},
      'out': {
        'sheets': {
          'sheet': {'field': {'name': 'sheet_url','kind': 'string','order': 2,'description': 'Feed Sheet URL','default': ''}},
          'tab': 'Weather',
          'range': 'A2:K',
          'delete': True
        }
      }
    }
  },
  {
    'lineitem_beta': {
      'auth': 'user',
      'read': {
        'sheet': {
          'sheet': {'field': {'name': 'sheet_url','kind': 'string','order': 2,'description': 'Feed Sheet URL','default': ''}},
          'tab': 'Rules',
          'range': 'A1:D'
        }
      },
      'patch': {
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
